# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 2 - Feature Extraction


In any practical machine learning problem, the data preparation and feature extraction stages are the most important and time-consuming. The final project exposes you to a real-world dataset. In this part of the final project, you are responsible to creating features that will be meaningful for prediction. Features are evaluated based on Information Gain, which you implemented in [Assignment 2](../assignment-2/assignment-2.ipynb).

Here is what will work well in this project:

* Extract some sample data, load it in [R](https://www.r-project.org), and do some intial analysis. Feel free to build models there to get a feel for the best features.
* Join the different tables--they are there for a reason. 
* Get creative.
* Read some of the Kaggle competition forums and kernels. 

Here is what will NOT work:

* Do not use only the features as provided in application_train.
* Do not try implementing new learning algorithm in order to generate features. If you find something that works, investigate what features were helpful and add the features. 
* Do not build lookup tables "embeddings" or other things you might have read about but were not covered in class. 
* Do not try to build a kernel matrix on all pairs. Re-evaluate the kernel instead.

In [170]:
require './assignment_lib'

false

In [171]:
dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true

#<SQLite3::Database:0x00000000088d74c8 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Question 1.1 (10 Points)

Implement ```create_dataset``` which runs an SQL query on a database and constructs a dataset like those we have used in this course. Add an ```id``` field for the ```SK_ID_CURR``` and store the ```TARGET``` in ```label```. 

If the query is:
```sql
select sk_id_curr, target, ext_source_1 from application_train  where ext_source_1 <> '' order by sk_id_curr;
```

then the result is:

```json
[
    {"label":1,"id":100002,"features":{"ext_source_1":0.08303696739132256}},
    {"label":0,"id":100015,"features":{"ext_source_1":0.7220444501416448}}
]
...
```
Note the features should not include the ID or Label. Feature keys should be lowercase and only contain keys fo which ```key.is_a? String``` returns true.



In [172]:
def create_dataset db, sql
  dataset = []
  db.execute sql do |row|
    # BEGIN YOUR CODE
    data = Hash.new  
    data["id"] = row["SK_ID_CURR"]
    data["label"] = row["TARGET"]
    data["features"] = Hash.new
    row.each_key do |key|
      if key != "SK_ID_CURR" and key != "TARGET" and key.is_a? String
        data["features"][key.downcase] = row[key]
      end
    end
    dataset<< data
  end
  return dataset
end

:create_dataset

In [173]:
def test_11()
  dataset = create_dataset $dev_db, "select sk_id_curr, target, ext_source_1 from application_train where ext_source_1 <> '' 
order by sk_id_curr limit 37"
  assert_equal 37, dataset.size
  assert_true(dataset[0]["features"].has_key? "ext_source_1")
  assert_equal(1, dataset[0]["features"].size)
  assert_equal(100002, dataset[0]["id"])  
  assert_in_delta(0.08303696, dataset[0]["features"]["ext_source_1"], 1e-4)
  assert_equal(1, dataset[0]["label"])    
end

test_11()

## Question 1.1 (20 points)

Copy and revise **your** information gain calculation for numeric and categorical features, from [Assignment 2](../assignment-2/assignment-2.ipynb). Copy the following implementations

* Class Distribution
* Entropy
* Information Gain after splitting
* Information gain for numerical features (fast version)


In [174]:

def class_distribution dataset
  ### BEGIN SOLUTION
  counts = Hash.new {|h,k| h[k] = 0}
  sum = 0.0
  
  dataset.each do |row| 
    counts[row["label"]] += 1
    sum += 1
  end
  
  counts.each_key {|k| counts[k] /= sum}
  return counts
  ### END SOLUTION
end

:class_distribution

In [175]:
def entropy dist
  ### BEGIN SOLUTION
  freq_x = dist.values
  total_freq = freq_x.inject(0.0) {|u,x| u += x}
  return 0.0 if total_freq <= 0.0
  
  prob_x = freq_x.collect {|f| f / total_freq}
  
  -prob_x.inject(0.0) do |u,p| 
    u += p > 0 ? p * Math.log(p) : 0.0
  end
  ### END SOLUTION
end

:entropy

In [176]:
def test_12_1()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  assert_in_delta(0.2686201883261589, h0, 1e-3)
end

test_12_1()

In [177]:
def information_gain h0, splits
  ### BEGIN SOLUTION
  total_size = splits.values.inject(0.0) {|u,v| u += v.size}
  split_entropy = splits.values.inject(0.0) do |u, v|
    p_v = class_distribution(v)
    h_c_v = entropy(p_v)
    u += (v.size / total_size) * h_c_v
  end
  h0 - split_entropy
  ### END SOLUTION
end

:information_gain

In [178]:
def test_12_2()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  splits = dataset.group_by {|row| row["features"]["flag_own_car"]}
  ig = information_gain h0, splits
  assert_in_delta(0.0002206258541794237, ig, 1e-4)
end

test_12_2()

In [179]:
def find_split_point_numeric y, h0, fname
  ### BEGIN SOLUTION
  ig_max = 0
  t_max = nil
  
  x = []
  y.each do |row|
    if row["features"][fname] !=""
      x << row
    end
  end
  
    

  feature_groups = x.group_by {|r| r["features"].fetch(fname, 0.0)}
  counts_right = Hash.new {|h,k| h[k] = 0}
  counts_left = Hash.new {|h,k| h[k] = 0}
  v_left = 0.0
  v_right = x.size.to_f

  feature_groups.each_key do |t|
    counts = Hash.new {|h,k| h[k] = 0}  
    feature_groups[t].each do |r| 
      counts[r["label"]] += 1
      counts_right[r["label"]] += 1
    end
    feature_groups[t] = counts
  end
  
  thresholds = feature_groups.keys.sort
  t = thresholds.shift
  
  feature_groups[t].each_key do |k| 
    counts_left[k] += feature_groups[t][k]
    counts_right[k] -= feature_groups[t][k]
    v_left += feature_groups[t][k]
    v_right -= feature_groups[t][k]
  end
  
  thresholds.each.with_index do |t, i|
    p_left = v_left / x.size
    p_right = v_right / x.size
    
    d_left = Hash.new
    d_right = Hash.new
    counts_left.each_key {|k| d_left[k] = counts_left[k] / v_left}
    counts_right.each_key {|k| d_right[k] = counts_right[k] / v_right}
        
    h_left = entropy(d_left)
    h_right = entropy(d_right)    
    ig = h0 - (p_left * h_left + p_right * h_right)
    if ig > ig_max
      ig_max = ig
      t_max = t
    end

    feature_groups[t].each_key do |k| 
      counts_left[k] += feature_groups[t][k]
      counts_right[k] -= feature_groups[t][k]
      v_left += feature_groups[t][k]
      v_right -= feature_groups[t][k]
    end
  end

  return [t_max, ig_max]
  ### END SOLUTION
end

:find_split_point_numeric

In [180]:
def test_12_3()
  # Check that there are three classes
  dataset = create_dataset $dev_db, "select target, sk_id_curr, ext_source_1, flag_own_car from application_train where ext_source_1 <> ''"
  dist = class_distribution dataset
  h0 = entropy dist
  
  t, ig = find_split_point_numeric dataset, h0, "ext_source_1"
  assert_in_delta(0.009751743140812785, ig, 1e-4)
end

test_12_3()

## Question 2.1 (70 Points)

Using whatever external software you want (hosted on your own devices), provide 15+ different features that have information >= 0.005. You may to implement several cells below, so please insert them above the test. 

Features must only be derived from the database but you are free to write whatever SQL queries you want. You may create temporary tables, but the database is read-only.

Pay close attention to the following aspects of feature design:

* Normalization: Z-score, L2, Min-Max, etc.
* Sparsity / missing values
* Frequency: Information is easily fooled by features with many values.
* Joins: Some of the best features in this dataset combine two columns from different tables.
* Transformations: One-hot, Binning, Discretization, Non-linear transformation

In [181]:
#Add extra cells as needed

In [182]:
    def ig_categorical_feature y, h0, fname
      x = []
      y.each do |r|
        if r["features"][fname] != ""
          x << r
        end
      end
      feature_groups = x.group_by {|r| r["features"].fetch(fname)}
      ig = information_gain h0,feature_groups
      return ig
    end

:ig_categorical_feature

In [183]:
def extract_features db
  dataset = []
  # BEGIN YOUR CODE
  sql1 = "select TARGET, SK_ID_CURR, OWN_CAR_AGE, EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, APARTMENTS_AVG, BASEMENTAREA_AVG, YEARS_BEGINEXPLUATATION_AVG, YEARS_BUILD_AVG, COMMONAREA_AVG, ELEVATORS_AVG, ENTRANCES_AVG, FLOORSMAX_AVG, FLOORSMIN_AVG, LANDAREA_AVG, LIVINGAPARTMENTS_AVG, LIVINGAREA_AVG, NONLIVINGAPARTMENTS_AVG, NONLIVINGAREA_AVG, APARTMENTS_MODE, BASEMENTAREA_MODE, YEARS_BEGINEXPLUATATION_MODE, YEARS_BUILD_MODE, COMMONAREA_MODE, ELEVATORS_MODE, ENTRANCES_MODE, FLOORSMAX_MODE, FLOORSMIN_MODE, LANDAREA_MODE, LIVINGAPARTMENTS_MODE, LIVINGAREA_MODE, NONLIVINGAPARTMENTS_MODE, NONLIVINGAREA_MODE, APARTMENTS_MEDI, BASEMENTAREA_MEDI, YEARS_BEGINEXPLUATATION_MEDI, YEARS_BUILD_MEDI, COMMONAREA_MEDI, ELEVATORS_MEDI, ENTRANCES_MEDI, FLOORSMAX_MEDI, FLOORSMIN_MEDI, LANDAREA_MEDI, LIVINGAPARTMENTS_MEDI, LIVINGAREA_MEDI, NONLIVINGAPARTMENTS_MEDI, NONLIVINGAREA_MEDI, FONDKAPREMONT_MODE, HOUSETYPE_MODE, TOTALAREA_MODE, WALLSMATERIAL_MODE, EMERGENCYSTATE_MODE, AMT_REQ_CREDIT_BUREAU_HOUR, AMT_REQ_CREDIT_BUREAU_DAY, AMT_REQ_CREDIT_BUREAU_WEEK, AMT_REQ_CREDIT_BUREAU_MON, AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_YEAR
            from application_train"
  dataset = create_dataset(db, sql1)
  #END YOUR CODE
  return dataset
end

:extract_features

In [184]:
extracted_dataset = extract_features($dev_db)
extracted_dataset[0]

{"id"=>100002, "label"=>1, "features"=>{"own_car_age"=>"", "ext_source_1"=>0.08303696739132256, "ext_source_2"=>0.2629485927471776, "ext_source_3"=>0.13937578009978951, "apartments_avg"=>0.0247, "basementarea_avg"=>0.0369, "years_beginexpluatation_avg"=>0.9722, "years_build_avg"=>0.6192, "commonarea_avg"=>0.0143, "elevators_avg"=>0, "entrances_avg"=>0.069, "floorsmax_avg"=>0.0833, "floorsmin_avg"=>0.125, "landarea_avg"=>0.0369, "livingapartments_avg"=>0.0202, "livingarea_avg"=>0.019, "nonlivingapartments_avg"=>0, "nonlivingarea_avg"=>0, "apartments_mode"=>0.0252, "basementarea_mode"=>0.0383, "years_beginexpluatation_mode"=>0.9722, "years_build_mode"=>0.6341, "commonarea_mode"=>0.0144, "elevators_mode"=>0, "entrances_mode"=>0.069, "floorsmax_mode"=>0.0833, "floorsmin_mode"=>0.125, "landarea_mode"=>0.0377, "livingapartments_mode"=>0.022, "livingarea_mode"=>0.0198, "nonlivingapartments_mode"=>0, "nonlivingarea_mode"=>0, "apartments_medi"=>0.025, "basementarea_medi"=>0.0369, "years_beginex

In [185]:
assert_not_nil extracted_dataset
assert_equal 15334, extracted_dataset.size
assert_true(extracted_dataset.all? {|row| row["features"].size >= 8}, "At least 6 non-zero features per row")
assert_true(extracted_dataset.flat_map {|row| row["features"].keys}.uniq.size >= 15,  "At least 15 features")

In [186]:
assert_equal 15334, extracted_dataset.collect {|row| row["id"]}.uniq.size
assert_equal 2, extracted_dataset.collect {|row| row["label"]}.uniq.size

h0 = entropy(class_distribution(extracted_dataset))
assert_in_delta(0.2797684909805576, h0, 1e-3)

In [187]:
features = extracted_dataset.flat_map {|row| row["features"].keys}.uniq
numeric_features = features.select {|k| extracted_dataset.reject {|row| row["features"][k] == ""}.all? {|row| row["features"].fetch(k, 0.0).is_a? Numeric}}

assert_true(numeric_features.size >= 4, "At least 4 numeric features")
def test_ig_numeric extracted_dataset, h0, test_feature1
  t, ig = find_split_point_numeric extracted_dataset, h0, test_feature1
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_numeric extracted_dataset, h0, numeric_features[0]

"own_car_age"

In [188]:
test_ig_numeric extracted_dataset, h0, numeric_features[1]

"ext_source_1"

In [189]:
test_ig_numeric extracted_dataset, h0, numeric_features[2]

"ext_source_2"

In [190]:
3.upto(numeric_features.size - 1) do |i|
  test_ig_numeric extracted_dataset, h0, numeric_features[i]
end

3

In [191]:
categorical_features = features.select {|k| extracted_dataset.all? {|row| row["features"].fetch(k, "").is_a? String}}

assert_true(categorical_features.size >= 4, "At least 4 categorical features")

def test_ig_categorical extracted_dataset, h0, test_feature1
  splits = extracted_dataset.group_by {|row| row["features"][test_feature1]}
  ig = information_gain h0, splits
  puts ig
  assert_true(ig >= 0.005, "Expected information gain for '#{test_feature1}' > 0.005")
  return test_feature1
end

test_ig_categorical extracted_dataset, h0, categorical_features[0]

0.00029929351724883313


Test::Unit::AssertionFailedError: Expected information gain for 'fondkapremont_mode' > 0.005.
<true> expected but was
<false>

In [192]:
test_ig_categorical extracted_dataset, h0, categorical_features[1]

0.0006166568172344955


Test::Unit::AssertionFailedError: Expected information gain for 'housetype_mode' > 0.005.
<true> expected but was
<false>

In [193]:
test_ig_categorical extracted_dataset, h0, categorical_features[2]

0.0007533832827185116


Test::Unit::AssertionFailedError: Expected information gain for 'wallsmaterial_mode' > 0.005.
<true> expected but was
<false>

In [194]:
3.upto(categorical_features.size - 1) do |i|
  test_ig_categorical extracted_dataset, h0, categorical_features[i]
end

0.00056517374815368


Test::Unit::AssertionFailedError: Expected information gain for 'emergencystate_mode' > 0.005.
<true> expected but was
<false>